Indian Lok Sabha Elections:
543 members (529 from states +13 from union territories) 

Max-strength in Lok Sabha 550 members 

Allocated members (530 (from 28 states)+20 (from union territories)) 

28- states, 8- union territories 

Lok Sabha elections --> Data set (2014 & 2019) --> separate (Andra Pradesh & Telangana) --> provide insights for Nova news media --> provide voter turnout trends which are less discussed  

In [2]:
!pip install --upgrade nbformat



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

Data loading:

In [4]:
df_2014 = pd.read_csv("./constituency_wise_results_2014.csv")
df_2019 = pd.read_csv("./constituency_wise_results_2019.csv")
state_codes = pd.read_csv("./dim_states_codes.csv")

print(df_2014.columns)
print(df_2019.columns)
print(state_codes.columns)

Index(['state', 'pc_name', 'candidate', 'sex', 'age', 'category', 'party',
       'party_symbol', 'general_votes', 'postal_votes', 'total_votes',
       'total_electors'],
      dtype='object')
Index(['state', 'pc_name', 'candidate', 'sex', 'age', 'category', 'party',
       'party_symbol', 'general_votes', 'postal_votes', 'total_votes',
       'total_electors'],
      dtype='object')
Index(['state_name', 'abbreviation'], dtype='object')


Data cleating:

In [5]:
df_2014.isnull().sum()
df_2019.isnull().sum()
state_codes.isnull().sum()

state_name      0
abbreviation    0
dtype: int64

In [6]:
def clean_text(x):
    if isinstance(x, str):
        return x.strip().title()
    return x

for col in ["pc_name", "state"]:
    if col in df_2014.columns:
        df_2014[col] = df_2014[col].apply(clean_text)
    if col in df_2019.columns:
        df_2019[col] = df_2019[col].apply(clean_text)

telangana_constituencies = [
    "Adilabad", "Nizamabad", "Karimnagar", "Medak", "Malkajgiri", "Secundrabad",
    "Hyderabad", "Chelvella", "Mahbubnagar", "Nagarkurnool", "Nalgonda",
    "Bhongir", "Warangal", "Mahabubabad", "Khammam", "Zahirabad"
]

df_2014.loc[
    df_2014["pc_name"].isin(telangana_constituencies), "state"
] = "Telangana"

Data transformation

In [7]:
df_2014["year"] = 2014
df_2019["year"] = 2019

df_all = pd.concat([df_2014, df_2019], ignore_index=True)

df_all = df_all.merge(state_codes, left_on="state", right_on="state_name", how="left")

state_summary = (
    df_all.groupby(["state", "year"])
    .agg({"total_votes": "sum"})
    .reset_index()
    .sort_values(["year", "state"])
)

party_summary = (
    df_all.groupby(["party", "year"])
    .agg({"total_votes": "sum", "pc_name": "count"})
    .rename(columns={"pc_name": "no_of_seats"})
    .reset_index()
)

df_all.to_csv("cleaned_combined_data.csv", index=False)
state_summary.to_csv("state_summary.csv", index=False)
party_summary.to_csv("party_summary.csv", index=False)

print(" Data cleaning and transformation completed successfully!")

 Data cleaning and transformation completed successfully!


In [8]:
df_all.isnull().sum()

state                0
pc_name              0
candidate            0
sex               1053
age               1053
category          1053
party                0
party_symbol       543
general_votes        0
postal_votes         0
total_votes          0
total_electors       0
year                 0
state_name         328
abbreviation       328
dtype: int64

| Term             | Level                            | Example             | Elects                          |
| ---------------- | -------------------------------- | ------------------- | ------------------------------- |
| **State**        | Large region                     | Telangana           | Many MPs (one per constituency) |
| **Constituency** | Small voting area inside a state | Chevella, Hyderabad | One MP(or)MLA per constituency         |


turnout_ratio = total_votes / total_electors, turnout ratio is to findout the actual participation of citizens
total_electors = total eligible voters in that constituency/state
total_votes = total votes actually cast

1 .Top 5 / Bottom 5 constituencies of 2014 & 2019 in terms of voter turnout ratio?

In [9]:
df_all["voter_turnout_ratio"] = df_all["total_votes"] / df_all["total_electors"]

turnout_const = df_all.groupby(['pc_name', 'year']).agg({'total_votes':'sum','total_electors':'sum'}).reset_index()
turnout_const['turnout_ratio'] = turnout_const['total_votes'] / turnout_const['total_electors']

top5_const_2014 = turnout_const[turnout_const['year']==2014].nlargest(5, 'turnout_ratio')
bottom5_const_2014 = turnout_const[turnout_const['year']==2014].nsmallest(5, 'turnout_ratio')

top5_const_2019 = turnout_const[turnout_const['year']==2019].nlargest(5, 'turnout_ratio')
bottom5_const_2019 = turnout_const[turnout_const['year']==2019].nsmallest(5, 'turnout_ratio')

print(top5_const_2014, bottom5_const_2014)
print(top5_const_2019, bottom5_const_2019)

            pc_name  year  total_votes  total_electors  turnout_ratio
992            Tura  2014       458071         1759503       0.260341
701        Nagaland  2014      1038910         4731888       0.219555
72   Arunachal East  2014       261291         1250816       0.208896
270     Daman & Diu  2014        87233          559135       0.156014
670         Mizoram  2014       433201         2808680       0.154237             pc_name  year  total_votes  total_electors  turnout_ratio
1020       Varanasi  2014      1030685        75958941       0.013569
236   Chennai South  2014      1071409        77218368       0.013875
36           Amethi  2014       874625        58444505       0.014965
234   Chennai North  2014       910114        58318072       0.015606
637      Malkajgiri  2014      1620444        98675573       0.016422
            pc_name  year  total_votes  total_electors  turnout_ratio
993            Tura  2019       583041         2867960       0.203295
702        Nagaland 

In [10]:
top_bottom_2014 = pd.concat([top5_const_2014, bottom5_const_2014]).sort_values('turnout_ratio')
top_bottom_2019 = pd.concat([top5_const_2019, bottom5_const_2019]).sort_values('turnout_ratio')
fig_2014 = px.bar(
    top_bottom_2014,
    x='turnout_ratio',
    y='pc_name',
    orientation='h',        
    color='turnout_ratio',   
    color_continuous_scale='Viridis',
    hover_data=['total_votes', 'total_electors'],
    title='Top and Bottom 5 Constituencies by Voter Turnout (2014)',
    labels={'pc_name':'Constituency', 'turnout_ratio':'Voter Turnout Ratio'}
)

fig_2014.update_layout(yaxis={'categoryorder':'total ascending'}) 
fig_2014.show()

fig_2019 = px.bar(
    top_bottom_2019,
    x='turnout_ratio',
    y='pc_name',
    orientation='h',         
    color='turnout_ratio',
    color_continuous_scale='Cividis',
    hover_data=['total_votes', 'total_electors'],
    title='Top and Bottom 5 Constituencies by Voter Turnout (2019)',
    labels={'pc_name':'Constituency', 'turnout_ratio':'Voter Turnout Ratio'}
)

fig_2019.update_layout(yaxis={'categoryorder':'total ascending'})
fig_2019.show()

2 .Top 5 / Bottom 5 states of 2014 & 2019 in terms of voter turnout ratio?

In [11]:
turnout_state = df_all.groupby(['state','year']).agg({'total_votes':'sum','total_electors':'sum'}).reset_index()
turnout_state['turnout_ratio'] = turnout_state['total_votes'] / turnout_state['total_electors']

top5_state_2014 = turnout_state[turnout_state['year']==2014].nlargest(5, 'turnout_ratio')
bottom5_state_2014 = turnout_state[turnout_state['year']==2014].nsmallest(5, 'turnout_ratio')

top5_state_2019 = turnout_state[turnout_state['year']==2019].nlargest(5, 'turnout_ratio')
bottom5_state_2019 = turnout_state[turnout_state['year']==2019].nsmallest(5, 'turnout_ratio')

print(top5_state_2014, bottom5_state_2014)
print(top5_state_2019, bottom5_state_2019)

          state  year  total_votes  total_electors  turnout_ratio
45     Nagaland  2014      1038910         4731888       0.219555
15  Daman & Diu  2014        87233          559135       0.156014
43      Mizoram  2014       433201         2808680       0.154237
33  Lakshadweep  2014        43239          349454       0.123733
56       Sikkim  2014       308967         2595390       0.119045               state  year  total_votes  total_electors  turnout_ratio
50       Puducherry  2014       740017        27942067       0.026484
47     Nct Of Delhi  2014      8271766       277906739       0.029765
21          Haryana  2014     11495151       384048106       0.029932
37      Maharashtra  2014     48717419      1605928601       0.030336
25  Jammu & Kashmir  2014      3566863       112439488       0.031723
                state  year  total_votes  total_electors  turnout_ratio
46           Nagaland  2019      1006357         6068885       0.165822
16        Daman & Diu  2019        87469

In [12]:
top_bottom_2014 = pd.concat([top5_state_2014, bottom5_state_2014]).sort_values('turnout_ratio')
top_bottom_2019 = pd.concat([top5_state_2019, bottom5_state_2019]).sort_values('turnout_ratio')

fig_2014 = px.bar(
    top_bottom_2014,
    x='turnout_ratio',
    y='state',
    orientation='h',          
    color='turnout_ratio',
    color_continuous_scale='Viridis',
    hover_data=['total_votes', 'total_electors'],
    title='Top and Bottom 5 States by Voter Turnout (2014)',
    labels={'state':'State', 'turnout_ratio':'Voter Turnout Ratio'}
)

fig_2014.update_layout(yaxis={'categoryorder':'total ascending'})  # sort bars
fig_2014.show()

fig_2019 = px.bar(
    top_bottom_2019,
    x='turnout_ratio',
    y='state',
    orientation='h',
    color='turnout_ratio',
    color_continuous_scale='Cividis',
    hover_data=['total_votes', 'total_electors'],
    title='Top and Bottom 5 States by Voter Turnout (2019)',
    labels={'state':'State', 'turnout_ratio':'Voter Turnout Ratio'}
)

fig_2019.update_layout(yaxis={'categoryorder':'total ascending'})
fig_2019.show()


Need to find the constituencies where the same political party won both in 2014 and 2019, and then sort those constituencies based on how strongly that winning party performed in 2019 their vote share percentage.
It is about how strong the winning party performed

3 .Which Constituencies have elected the same party for two consicutive elections,rank them by % of votes to that winning party in 2019?

In [13]:
df_2014 = df_all[df_all['year'] == 2014]
df_2019 = df_all[df_all['year'] == 2019]

winners_2014 = df_2014.groupby('pc_name').agg({'party': 'first', 'total_votes': 'sum', 'total_electors': 'sum'}).reset_index()
winners_2019 = df_2019.groupby('pc_name').agg({'party': 'first', 'total_votes': 'sum', 'total_electors': 'sum'}).reset_index()

merged_winners = pd.merge(winners_2014, winners_2019, on='pc_name', suffixes=('_2014', '_2019'))

same_party_winners = merged_winners[merged_winners['party_2014'] == merged_winners['party_2019']]

same_party_winners['vote_pct_2019'] = (same_party_winners['total_votes_2019'] / same_party_winners['total_electors_2019']) * 100

ranked_constituencies = same_party_winners[['pc_name', 'party_2019', 'vote_pct_2019']].sort_values(by='vote_pct_2019', ascending=False)

print(ranked_constituencies.head(10))

               pc_name party_2019  vote_pct_2019
36      Arunachal East        INC      14.410014
126        Daman & Diu        INC      14.369805
280             Ladakh        BJP      14.107525
87            Bhilwara        INC      13.122662
61            Banswara        BJP      12.146859
163  Gadchiroli-Chimur        BJP      12.043364
9              Alathur        BSP      11.488428
60            Bansgaon        BJP      11.067884
356          North Goa        INC      10.997999
244             Kanthi       AITC      10.723802


C:\Users\16052\AppData\Local\Temp\ipykernel_19360\3754989790.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [14]:
top10 = ranked_constituencies.head(10)

fig = px.bar(
    top10,
    x='vote_pct_2019',
    y='pc_name',
    orientation='h',
    color='vote_pct_2019',            
    color_continuous_scale='Oranges',
    hover_data=['party_2019'],
    title='Top 10 Constituencies that Re-elected the Same Party (2014-2019)',
    labels={'pc_name':'Constituency', 'vote_pct_2019':'Vote Percentage (2019)'}
)

fig.update_layout(yaxis={'categoryorder':'total descending'})
fig.show()

constituencies who have voted for different parties in two elections of 2014 and 2019. Areas where voters shifted their support from one party to another.

4 .Which constituencies have voted for different parties in two elections(list top 10 based on difference (2014-2019) in winner vote percentage in two elections).

In [15]:
df_2014 = df_all[df_all['year'] == 2014].copy()
df_2019 = df_all[df_all['year'] == 2019].copy()

df_2014['vote_pct'] = (df_2014['general_votes'] / df_2014['total_votes']) * 100
df_2019['vote_pct'] = (df_2019['general_votes'] / df_2019['total_votes']) * 100

winners_2014 = df_2014.loc[df_2014.groupby('pc_name')['general_votes'].idxmax(), 
                           ['pc_name', 'party', 'vote_pct']]
winners_2019 = df_2019.loc[df_2019.groupby('pc_name')['general_votes'].idxmax(), 
                           ['pc_name', 'party', 'vote_pct']]

merged_winners = pd.merge(winners_2014, winners_2019, on='pc_name', suffixes=('_2014', '_2019'))

diff_party = merged_winners[merged_winners['party_2014'] != merged_winners['party_2019']].copy()

diff_party['vote_pct_diff'] = abs(diff_party['vote_pct_2014'] - diff_party['vote_pct_2019'])

top10_diff = diff_party.sort_values(by='vote_pct_diff', ascending=False).head(10)

print(top10_diff[['pc_name', 'party_2014', 'vote_pct_2014', 'party_2019', 'vote_pct_2019', 'vote_pct_diff']])

            pc_name party_2014  vote_pct_2014 party_2019  vote_pct_2019  \
280          Ladakh        IND      99.916331        BJP      96.273943   
36   Arunachal East        INC      98.220421        BJP      96.545427   
407          Rohtak        INC      99.966739        BJP      98.675078   
494    Virudhunagar       ADMK      99.886401        INC      98.679927   
456         Tenkasi       ADMK      99.864103        DMK      98.779812   
136      Dharmapuri        PMK      99.910507        DMK      98.838584   
0          Adilabad        TRS      98.819767        BJP      99.872275   
434          Sikkim        SDF      97.560141        SKM      96.537904   
177  Gopalganj (Sc)        BJP      99.996867      JD(U)      98.992168   
467     Tirunelveli       ADMK      99.909077        DMK      98.966717   

     vote_pct_diff  
280       3.642388  
36        1.674994  
407       1.291661  
494       1.206474  
456       1.084291  
136       1.071923  
0         1.052509  
434   

In [16]:
top10_diff = diff_party.sort_values(by='vote_pct_diff', ascending=False).head(10)

fig = px.bar(
    top10_diff,
    x='vote_pct_diff',
    y='pc_name',
    orientation='h',
    color='vote_pct_diff',  
    color_continuous_scale='Reds',
    hover_data=['party_2014', 'vote_pct_2014', 'party_2019', 'vote_pct_2019'],
    title='Top 10 Constituencies That Switched Parties (2014 - 2019)',
    labels={'pc_name':'Constituency', 'vote_pct_diff':'Difference in Winning Vote %'}
)

fig.update_layout(yaxis={'categoryorder':'total descending'})
fig.show()

The margin of a candidate is the difference between the votes received by the winner and the votes received by the runner-up in an election.

5 .Top 5 candidates based on margin difference with runners in 2014 and 2019?

In [17]:
def calculate_margin(df):
    df_sorted = df.sort_values(['pc_name', 'general_votes'], ascending=[True, False])
    
    top2 = df_sorted.groupby('pc_name').head(2)
    
    margin = top2.groupby('pc_name').apply(
        lambda x: x.iloc[0]['general_votes'] - x.iloc[1]['general_votes']
        if len(x) > 1 else x.iloc[0]['general_votes']
    )
    
    winners = df_sorted.groupby('pc_name').first().reset_index()
    
    winners['margin_votes'] = margin.values
    
    return winners[['pc_name', 'party', 'candidate', 'general_votes', 'margin_votes']]
winners_2014 = calculate_margin(df_2014)
winners_2019 = calculate_margin(df_2019)

top5_2014 = winners_2014.sort_values(by='margin_votes', ascending=False).head(5)
top5_2019 = winners_2019.sort_values(by='margin_votes', ascending=False).head(5)

print("Top 5 candidates in 2014 by winning margin:\n", top5_2014)
print("\nTop 5 candidates in 2019 by winning margin:\n", top5_2019)


Top 5 candidates in 2014 by winning margin:
        pc_name party                candidate  general_votes  margin_votes
490   Vadodara   BJP            NARENDRA MODI         841592        568003
175  Ghaziabad   BJP        VIJAY KUMAR SINGH         758113        566925
354    Navsari   BJP              C. R. PATIL         818367        556519
212     Jaipur   BJP         RAMCHARAN BOHARA         859973        536942
457      Surat   BJP  DARSHANA VIKRAM JARDOSH         717429        532525

Top 5 candidates in 2019 by winning margin:
        pc_name party                candidate  general_votes  margin_votes
379    Navsari   BJP              C. R. Patil         969430        687767
272     Karnal   BJP            Sanjay Bhatia         909432        654269
171  Faridabad   BJP              KRISHAN PAL         910787        636032
94    Bhilwara   BJP  SUBHASH CHANDRA BAHERIA         936065        610920
522   Vadodara   BJP          RANJANBEN BHATT         880905        587825


C:\Users\16052\AppData\Local\Temp\ipykernel_19360\836391305.py:6: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

C:\Users\16052\AppData\Local\Temp\ipykernel_19360\836391305.py:6: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



A large margin indicates a strong victory, while a small margin means a close contest.

In [58]:
top5_all = pd.concat([top5_2014, top5_2019], ignore_index=True)
facest_col = np.where(top5_all.index < 5, 2014, 2019)
top5_all['year'] = facest_col
fig = px.bar(
    top5_all,
    x='candidate',
    y='margin_votes',
    color='party',
    barmode='group',
    text='pc_name',
    facet_col='year',
    title='Top 5 Candidates by Winning Margin (2014 vs 2019)',
    color_discrete_sequence=px.colors.qualitative.Bold
)

# Improve appearance
fig.update_traces(textposition='outside')
fig.update_layout(
    xaxis_title='Candidate Name',
    yaxis_title='Winning Margin (Votes)',
    showlegend=True,
    height=600
)

fig.show()


Since we want national-level, we ignore constituency and state names and just group by year and party.

6 .% split of votes of parties between 2014 vs 2019 at national level?

In [59]:
df_all = pd.concat([df_2014, df_2019], ignore_index=True)

# Step 1: Aggregate total votes per party per year
party_votes_national = df_all.groupby(['year', 'party']).agg({'general_votes': 'sum'}).reset_index()

# Step 2: Calculate % of total votes per year
party_votes_national['vote_pct'] = party_votes_national.groupby('year')['general_votes'].transform(lambda x: (x / x.sum()) * 100)

# Step 3: Sort for clarity
party_votes_national = party_votes_national.sort_values(['year', 'vote_pct'], ascending=[True, False])

# Step 4: Display
print(party_votes_national)


      year party  general_votes   vote_pct
80    2014   BJP      160763427  30.968387
163   2014   INC       96486943  18.586597
110   2014   BSP       22417360   4.318330
31    2014  AITC       21185271   4.080988
385   2014    SP       18623577   3.587521
...    ...   ...            ...        ...
651   2019  DKSP            337   0.000055
1074  2019   ToP            330   0.000054
646   2019  CoAP            275   0.000045
650   2019  DJHP            273   0.000045
801   2019  MLJK            265   0.000043

[1127 rows x 4 columns]


In [60]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df_pivot = party_votes_national.pivot(index='party', columns='year', values='vote_pct').reset_index()

fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=False,
    vertical_spacing=0.05,
    specs=[[{"type": "table"}],
           [{"type": "bar"}],
           [{"type": "bar"}]]
)
fig.add_trace(
    go.Table(
        header=dict(
            values=["Party", "2014 Vote Share (%)", "2019 Vote Share (%)"],
            font=dict(size=12, color='white'),
            fill_color='darkblue',
            align='center'
        ),
        cells=dict(
            values=[df_pivot['party'], df_pivot[2014], df_pivot[2019]],
            align='center'
        )
    ),
    row=1, col=1
)
fig.add_trace(
    go.Bar(
        x=df_pivot['party'],
        y=df_pivot[2014],
        text=df_pivot[2014],
        textposition='outside',
        marker_color='orange',
        name='2014'
    ),
    row=2, col=1
)
fig.add_trace(
    go.Bar(
        x=df_pivot['party'],
        y=df_pivot[2019],
        text=df_pivot[2019],
        textposition='outside',
        marker_color='green',
        name='2019'
    ),
    row=3, col=1
)
fig.update_layout(
    title_text="National Vote Share of Parties (2014 vs 2019)",
    height=1200,
    width=1200,
    showlegend=False,
    plot_bgcolor='white',
    paper_bgcolor='white'
)
fig.show()

votes for each party got in the years of 2014 and 2019 in state level of % split them.

7 .% split of votes of parties between 2014 vs 2019 at state level?

In [21]:
state_party_votes = df_all.groupby(['year', 'state', 'party']).agg({'general_votes':'sum'}).reset_index()

state_party_votes['vote_pct'] = state_party_votes.groupby(['year', 'state'])['general_votes'].transform(lambda x: (x / x.sum()) * 100)

state_party_votes = state_party_votes.sort_values(['year', 'state', 'vote_pct'], ascending=[True, True, False])

print(state_party_votes.head(20))

    year                      state       party  general_votes   vote_pct
3   2014  Andaman & Nicobar Islands         BJP          90882  47.779069
7   2014  Andaman & Nicobar Islands         INC          83144  43.710998
0   2014  Andaman & Nicobar Islands        AAAP           3725   1.958331
8   2014  Andaman & Nicobar Islands         IND           3511   1.845825
2   2014  Andaman & Nicobar Islands        AITC           2283   1.200233
6   2014  Andaman & Nicobar Islands         CPM           1776   0.933690
10  2014  Andaman & Nicobar Islands        NOTA           1563   0.821710
9   2014  Andaman & Nicobar Islands         NCP           1150   0.604585
4   2014  Andaman & Nicobar Islands         BSP           1139   0.598802
5   2014  Andaman & Nicobar Islands  CPI(ML)(L)            436   0.229217
11  2014  Andaman & Nicobar Islands          SP            379   0.199250
1   2014  Andaman & Nicobar Islands        AIFB            225   0.118288
52  2014             Andhra Pradesh   

In [22]:
states = state_party_votes['state'].unique()
chunks = [states[i::4] for i in range(4)]

for i, chunk in enumerate(chunks, 1):
    df_chunk = state_party_votes[state_party_votes['state'].isin(chunk)]
    
    fig = px.bar(
        df_chunk,
        x='state',
        y='vote_pct',
        color='party',
        facet_col='year',
        text='vote_pct',
        title=f'Vote Share of Parties by State (Chunk {i})',
        labels={'vote_pct':'Vote Percentage (%)', 'state':'State'}
    )
    
    fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
    fig.update_layout(
        width=1200,
        height=600,
        xaxis_tickangle=-45,
        showlegend=True
    )
    
    fig.show()


computes vote share per constituency in 2014 and 2019

8 .List top 5 constituencies for two major national parties where they have gained votes share in 2019 as compared to 2014?

In [23]:
df_all.columns = df_all.columns.str.strip().str.lower()

print("Columns in df_all:", df_all.columns.tolist())

df_all['general_votes'] = pd.to_numeric(df_all['general_votes'], errors='coerce').fillna(0)

total_votes = (
    df_all.groupby(['year', 'pc_name'])['general_votes']
    .sum()
    .reset_index()
    .rename(columns={'general_votes': 'total_votes'})
)

df = df_all.merge(total_votes, on=['year', 'pc_name'], how='left')

print("Columns in merged df:", df.columns.tolist())

df['vote_share'] = (df['general_votes'] / df['total_votes_x']) * 100

party_share = (
    df.groupby(['year', 'pc_name', 'party'])['vote_share']
    .sum()
    .reset_index()
)

pivot = (
    party_share
    .pivot_table(index=['pc_name', 'party'], columns='year', values='vote_share')
    .reset_index()
)
pivot.columns.name = None
pivot = pivot.rename(columns={2014: 'share_2014', 2019: 'share_2019'})
pivot = pivot.fillna(0)

pivot['diff'] = pivot['share_2019'] - pivot['share_2014']

major_parties = ['BJP', 'INC']
pivot = pivot[pivot['party'].isin(major_parties)]

top5 = (
    pivot[pivot['diff'] > 0]
    .sort_values(['party', 'diff'], ascending=[True, False])
    .groupby('party')
    .head(5)
    .reset_index(drop=True)
)

print("Top 5 constituencies where each party gained vote share in 2019 vs 2014:")
print(top5[['party', 'pc_name', 'share_2014', 'share_2019', 'diff']])


Columns in df_all: ['state', 'pc_name', 'candidate', 'sex', 'age', 'category', 'party', 'party_symbol', 'general_votes', 'postal_votes', 'total_votes', 'total_electors', 'year', 'state_name', 'abbreviation', 'voter_turnout_ratio']
Columns in merged df: ['state', 'pc_name', 'candidate', 'sex', 'age', 'category', 'party', 'party_symbol', 'general_votes', 'postal_votes', 'total_votes_x', 'total_electors', 'year', 'state_name', 'abbreviation', 'voter_turnout_ratio', 'total_votes_y']
Top 5 constituencies where each party gained vote share in 2019 vs 2014:
  party                 pc_name  share_2014  share_2019        diff
0   BJP  Dadra And Nagar Haveli         0.0   99.955785   99.955785
1   BJP                Jaynagar         0.0   99.912922   99.912922
2   BJP             Nabarangpur         0.0   99.904037   99.904037
3   BJP                Adilabad         0.0   99.872275   99.872275
4   BJP              Malkajgiri         0.0   99.858355   99.858355
5   INC  Dadra And Nagar Haveli    

In [24]:
top5 = top5.sort_values(['party', 'diff'], ascending=[True, False])

fig = px.bar(
    top5,
    x='diff',
    y='pc_name',
    color='party',
    orientation='h',
    facet_col='party',
    text='diff',
    title='Top 5 Constituencies where BJP & INC Gained Vote Share (2019 vs 2014)',
    labels={'diff': 'Vote Share Gain (in % points)', 'pc_name': 'Constituency'},
    color_discrete_map={'BJP': 'teal', 'INC': 'pink'} 
)

fig.update_traces(
    texttemplate='%{text:.2f}%',
    textposition='outside'
)
fig.update_layout(
    height=550,
    showlegend=False,
    title_font_size=18,
    font=dict(size=13, color='black'),
    xaxis_title_font=dict(size=14),
    yaxis_title_font=dict(size=14),
    plot_bgcolor='white',     
    paper_bgcolor='white',    
    bargap=0.4,
)

fig.for_each_yaxis(lambda yaxis: yaxis.update(title_text=''))
fig.for_each_xaxis(lambda xaxis: xaxis.update(title_text='Vote Share Gain (%)'))
fig.show()

That means we’ll look for negative differences (share_2019 - share_2014 < 0) and find the largest losses.

9 .List top 5 constituencies for two major national parties where they have lost votes share in 2019 as comared to 2014?

In [25]:
major_parties = ['BJP', 'INC']
pivot_loss = pivot[pivot['party'].isin(major_parties)]

pivot_loss = pivot_loss[pivot_loss['diff'] < 0]

top5_loss = (
    pivot_loss
    .sort_values(['party', 'diff'], ascending=[True, True]) 
    .groupby('party')
    .head(5)
    .reset_index(drop=True)
)

print("Top 5 constituencies where each party LOST vote share in 2019 vs 2014:\n")
print(top5_loss[['party', 'pc_name', 'share_2014', 'share_2019', 'diff']])


Top 5 constituencies where each party LOST vote share in 2019 vs 2014:

  party               pc_name  share_2014  share_2019        diff
0   BJP  Dadar & Nagar Haveli  100.000000    0.000000 -100.000000
1   BJP               Giridih   99.997959    0.000000  -99.997959
2   BJP                Supaul   99.997597    0.000000  -99.997597
3   BJP                 Siwan   99.997048    0.000000  -99.997048
4   BJP        Gopalganj (Sc)   99.996867    0.000000  -99.996867
5   INC            Aurangabad  199.908283   99.889965 -100.018318
6   INC  Dadar & Nagar Haveli  100.000000    0.000000 -100.000000
7   INC             Firozabad  100.000000    0.000000 -100.000000
8   INC               Kodarma  100.000000    0.000000 -100.000000
9   INC               Bijapur   99.999502    0.000000  -99.999502


In [26]:
top5_loss = top5_loss.sort_values(['party', 'diff'], ascending=[True, True])

fig = px.bar(
    top5_loss,
    x='diff',
    y='pc_name',
    color='party',
    orientation='h',
    facet_col='party',
    text='diff',
    title='Top 5 Constituencies where BJP & INC Lost Vote Share (2019 vs 2014)',
    labels={'diff': 'Vote Share Change (in % points)', 'pc_name': 'Constituency'},
    color_discrete_map={'BJP': 'rosybrown', 'INC': 'turquoise'}  
)

fig.update_traces(
    texttemplate='%{text:.2f}%',
    textposition='outside'
)
fig.update_layout(
    height=550,
    showlegend=False,
    title_font_size=18,
    font=dict(size=13, color='black'),
    xaxis_title_font=dict(size=14),
    yaxis_title_font=dict(size=14),
    plot_bgcolor='sky blue',
    paper_bgcolor='white',
    bargap=0.4,
)
fig.for_each_yaxis(lambda yaxis: yaxis.update(title_text=''))
fig.for_each_xaxis(lambda xaxis: xaxis.update(title_text='Vote Share Change (%)'))
fig.show()

most votes nota constituency

10 .Which constituency has voted the most for NOTA?

In [27]:
nota_votes = df_all[df_all['party'].str.upper() == 'NOTA'].copy()

nota_votes_sum = nota_votes.groupby('pc_name')['general_votes'].sum().reset_index()

max_nota = nota_votes_sum.loc[nota_votes_sum['general_votes'].idxmax()]

print(f"Constituency that voted the most for NOTA: {max_nota['pc_name']}")
print(f"Number of NOTA votes: {int(max_nota['general_votes'])}")

Constituency that voted the most for NOTA: Gopalganj (Sc)
Number of NOTA votes: 69406


In [28]:
top10_nota = nota_votes_sum.sort_values('general_votes', ascending=False).head(10)

fig = px.bar(
    top10_nota,
    x='general_votes',
    y='pc_name',
    orientation='h',
    text='general_votes',
    title='Top 10 Constituencies Voting for NOTA',
    labels={'general_votes': 'NOTA Votes', 'pc_name': 'Constituency'},
    color_discrete_sequence=['purple'] 
)
fig.update_traces(texttemplate='%{text}', textposition='outside')
fig.update_layout(
    height=500,
    plot_bgcolor='white',
    paper_bgcolor='white',
    title_font_size=18,
    yaxis=dict(autorange='reversed'), 
)
fig.show()

want to find the constituencies in 2019 where the winning candidate’s party had less than 10% of the total votes at the state level.

11 .Which constituencies have elected candidates whose party has less than 10% vote share to state level in 2019?

In [29]:
df_2019 = df_all[df_all['year'] == 2019].copy()

state_total_votes = df_2019.groupby('state')['general_votes'].sum().reset_index().rename(columns={'general_votes':'state_total_votes'})

party_state_votes = df_2019.groupby(['state','party'])['general_votes'].sum().reset_index()

party_state_votes = party_state_votes.merge(state_total_votes, on='state', how='left')
party_state_votes['state_vote_share'] = (party_state_votes['general_votes'] / party_state_votes['state_total_votes']) * 100

low_share_parties = party_state_votes[party_state_votes['state_vote_share'] < 10][['state','party']]

winners = df_2019.loc[df_2019.groupby('pc_name')['general_votes'].idxmax()]

winners_low_share = winners.merge(low_share_parties, on=['state','party'], how='inner')

result = winners_low_share[['state','pc_name','party','general_votes']]
print("Constituencies elected candidates whose party had <10% vote share at state level in 2019:\n")
print(result)

Constituencies elected candidates whose party had <10% vote share at state level in 2019:

              state          pc_name   party  general_votes
0       Maharashtra         Amravati     IND         507844
1   Jammu & Kashmir         Anantnag     JKN          40032
2       West Bengal       Baharampur     INC         589580
3   Jammu & Kashmir        Baramulla     JKN         132969
4        Tamil Nadu      Chidambaram     VCK         498401
5        Tamil Nadu       Coimbatore    CPIM         567741
6             Assam           Dhubri   AIUDF         718131
7         Jharkhand          Giridih   AJSUP         646864
8            Punjab        Gurdaspur     BJP         551177
9             Bihar     Hajipur (Sc)     LJP         539665
10        Karnataka           Hassan   JD(S)         675512
11           Punjab       Hoshiarpur     BJP         416711
12        Telangana        Hyderabad   AIMIM         517239
13            Bihar       Jamui (Sc)     LJP         527860
14       

In [30]:
plot_df = winners_low_share.sort_values('general_votes', ascending=True)

fig = px.bar(
    plot_df,
    x='general_votes',
    y='pc_name',
    color='party',
    orientation='h',
    text='general_votes',
    title='Constituencies where winning party had <10% state-level vote share (2019)',
    labels={'general_votes':'Votes Received', 'pc_name':'Constituency'},
    color_discrete_sequence=px.colors.qualitative.Pastel  
)

fig.update_traces(texttemplate='%{text}', textposition='outside')

fig.update_layout(
    height=600,
    plot_bgcolor='white',
    paper_bgcolor='white',
    title_font_size=18,
    yaxis=dict(autorange='reversed'), 
)
fig.show()

Highlighting states with increased engagement can indicate growing political awareness or successful voter mobilization efforts.

12. Which states showed the highest increase in voter turnout between 2014 and 2019?

In [31]:
state_turnout = df_all.groupby(['state', 'year'])['voter_turnout_ratio'].mean().reset_index()

state_turnout_pivot = state_turnout.pivot(index='state', columns='year', values='voter_turnout_ratio').reset_index()
state_turnout_pivot['diff'] = state_turnout_pivot[2019] - state_turnout_pivot[2014]

top_increase = state_turnout_pivot.sort_values('diff', ascending=False).head(10)
print("top_increase:\n", top_increase)

top_increase:
 year           state      2014      2019      diff
9                Goa  0.073400  0.107336  0.033936
26        Puducherry  0.026484  0.042756  0.016272
21         Meghalaya  0.117765  0.133442  0.015677
34       Uttarakhand  0.038420  0.053200  0.014779
28         Rajasthan  0.045666  0.060396  0.014729
16            Kerala  0.051189  0.063011  0.011822
33     Uttar Pradesh  0.034258  0.044772  0.010514
3              Assam  0.063450  0.071434  0.007984
1     Andhra Pradesh  0.054331  0.058382  0.004051
14         Jharkhand  0.035247  0.038669  0.003421


In [32]:
fig = px.scatter(
    top_increase,
    x='diff',
    y='state',
    size='diff',              
    color='diff',             
    text='diff',              
    title='Top 10 States with Highest Increase in Voter Turnout (2014-2019)',
    labels={'diff':'Increase in Turnout (%)','state':'State'},
    color_continuous_scale='RdPu',  
    size_max=80            
)
fig.update_traces(textposition='top center')

fig.update_layout(
    plot_bgcolor='white', 
    paper_bgcolor='white', 
    width=1400,           
    height=800,            
    yaxis=dict(
        autorange='reversed', 
        showline=True,
        linecolor='black', 
        linewidth=2
    ),
    xaxis=dict(
        showline=True,      
        linecolor='black', 
        linewidth=2
    )
)
fig.show()


Identifying declines can help understand regions where political engagement may be dropping, revealing potential areas for awareness campaigns.

13. Which states showed the largest decline in voter turnout between 2014 and 2019?


In [33]:
top_decline = state_turnout_pivot.sort_values('diff').head(10)
print("top_decline:\n", top_decline)

top_decline:
 year              state      2014      2019      diff
22              Mizoram  0.154237  0.090066 -0.064170
23             Nagaland  0.219555  0.165822 -0.053733
29               Sikkim  0.119045  0.067520 -0.051525
5            Chandigarh  0.040948  0.019080 -0.021868
31            Telangana  0.042985  0.023801 -0.019184
8           Daman & Diu  0.156014  0.143698 -0.012316
2     Arunachal Pradesh  0.122086  0.116943 -0.005144
27               Punjab  0.034533  0.029424 -0.005108
13      Jammu & Kashmir  0.036549  0.031848 -0.004701
24         Nct Of Delhi  0.029070  0.024774 -0.004295


In [34]:
fig = px.scatter(
    top_decline,
    x='diff',
    y='state',
    size=top_decline['diff'].abs(),   
    color=top_decline['diff'].abs(),  
    text='diff',
    title='Top 10 States with Largest Decline in Voter Turnout (2014→2019)',
    labels={'diff':'Decrease in Turnout (%)','state':'State'},
    color_continuous_scale='PuRd',    
    size_max=80
)
fig.update_traces(textposition='top center')
fig.update_layout(
    plot_bgcolor='white',
    paper_bgcolor='white',
    width=1200,
    height=700,
    yaxis=dict(
        autorange='reversed',
        showline=True,
        linecolor='black',
        linewidth=2
    ),
    xaxis=dict(
        showline=True,
        linecolor='black',
        linewidth=2
    )
)
fig.show()

Shows where elections were closely contested, highlighting regions of high political uncertainty.

14. Which constituencies had the most competitive elections (smallest winning margins) in 2014 and 2019?

In [37]:
df_sorted = df_all.sort_values(['pc_name','general_votes'], ascending=[True, False])
top2 = df_sorted.groupby('pc_name').head(2)
margin = top2.groupby('pc_name').apply(lambda x: x.iloc[0]['general_votes'] - x.iloc[1]['general_votes']).reset_index(name='margin')
competitive = margin.sort_values('margin').head(10)
print("\nMost competitive constituencies (smallest winning margin):\n", competitive)


Most competitive constituencies (smallest winning margin):
                        pc_name  margin
320               Machhlishahr     330
116             Chamarajanagar     577
257                    Kairana     691
314                Lakshadweep     816
228                  Jahanabad    1075
292                     Khunti    1098
475                     Sikkim    1439
28   Andaman & Nicobar Islands    1477
69          Bardhaman Durgapur    1839
349                     Meerut    2379


C:\Users\16052\AppData\Local\Temp\ipykernel_19360\3463632794.py:3: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [38]:
import plotly.graph_objects as go
import plotly.figure_factory as ff

table_data = [['Constituency', 'Winning Margin']]
for i, row in competitive.iterrows():
    table_data.append([row['pc_name'], row['margin']])

fig = ff.create_table(table_data, height_constant=50)

trace = go.Bar(
    x=competitive['margin'],
    y=competitive['pc_name'],
    orientation='h',
    text=competitive['margin'],
    textposition='auto',
    textfont=dict(size=14, color='black', family='Arial'),
    marker=dict(color='maroon'),
    name='Winning Margin',
    xaxis='x2',
    yaxis='y2'
)
fig.add_traces([trace])
fig.layout['xaxis2'] = {}
fig.layout['yaxis2'] = {}

fig.layout.yaxis.update({'domain': [0, 0.35]})
fig.layout.yaxis2.update({'domain': [0.45, 1], 'anchor': 'x2'})
fig.layout.xaxis2.update({'anchor': 'y2', 'title': 'Winning Margin'})

fig.update_layout(
    title=dict(text='Top 10 Most Competitive Constituencies', x=0.5, xanchor='center', yanchor='top'),
    title_font=dict(size=28, family='Arial', color='black'),
    height=1500,
    width=1200,
    margin=dict(t=150, l=50, r=50, b=50), 
    plot_bgcolor='white',
    paper_bgcolor='white'
)
fig.show()

Indicates dynamic political behavior and voter swing regions that may be overlooked in mainstream analysis.

15. Which constituencies had the largest shift in vote share between 2014 and 2019 for any party?

In [39]:
party_share = df_all.groupby(['year','pc_name','party'])['general_votes'].sum().reset_index()
party_share['vote_share'] = party_share.groupby(['year','pc_name'])['general_votes'].transform(lambda x: x/x.sum()*100)
pivot = party_share.pivot_table(index=['pc_name','party'], columns='year', values='vote_share').fillna(0)
pivot['diff'] = (pivot[2019] - pivot[2014]).abs()
top_shift = pivot.sort_values('diff', ascending=False).head(10).reset_index()
print("\nConstituencies with biggest shifts in party vote shares (2014-2019):\n", top_shift)


Constituencies with biggest shifts in party vote shares (2014-2019):
 year       pc_name party       2014       2019       diff
0         Nagaland   NPF  68.655121   0.000000  68.655121
1     Tripura East   CPM  65.655731   0.000000  65.655731
2          Bikaner   BJP  62.877117   0.000000  62.877117
3     Tripura West   CPM  62.689925   0.000000  62.689925
4             Durg   BJP   0.000000  60.995306  60.995306
5            Karur   INC   2.914192  62.982822  60.068629
6           Raipur   BJP   0.000000  60.016967  60.016967
7     Bikaner (Sc)   BJP   0.000000  59.818216  59.818216
8         Jaynagar  AITC   0.000000  56.141291  56.141291
9           Nagaur  RLTP   0.000000  54.805708  54.805708


In [40]:
fig = px.bar(top_shift, x='diff', y='pc_name', color='party', orientation='h', text='diff',
             title='Top 10 Constituencies with Largest Vote Share Shift (2014-2019)',
             labels={'diff':'Vote Share Change (%)','pc_name':'Constituency'})
fig.update_layout(yaxis=dict(autorange='reversed'), plot_bgcolor='white', paper_bgcolor='white')
fig.show()

Highlights the role of regional or small parties and voter diversity, which is often ignored in high-level election coverage.

16. Which constituencies elected candidates from parties with low overall vote share at the state level?


In [41]:
state_total = df_all[df_all['year']==2019].groupby(['state','party'])['general_votes'].sum().reset_index()
state_total['state_share'] = state_total.groupby('state')['general_votes'].transform(lambda x: x/x.sum()*100)
low_share_parties = state_total[state_total['state_share']<10]

winners = df_all[df_all['year']==2019].loc[df_all[df_all['year']==2019].groupby('pc_name')['general_votes'].idxmax()]
winners_low_share = winners.merge(low_share_parties[['state','party']], on=['state','party'], how='inner')
print("\nConstituencies won by parties with <10% state-level vote share in 2019:\n", winners_low_share[['state','pc_name','party','general_votes']])


Constituencies won by parties with <10% state-level vote share in 2019:
               state          pc_name   party  general_votes
0       Maharashtra         Amravati     IND         507844
1   Jammu & Kashmir         Anantnag     JKN          40032
2       West Bengal       Baharampur     INC         589580
3   Jammu & Kashmir        Baramulla     JKN         132969
4        Tamil Nadu      Chidambaram     VCK         498401
5        Tamil Nadu       Coimbatore    CPIM         567741
6             Assam           Dhubri   AIUDF         718131
7         Jharkhand          Giridih   AJSUP         646864
8            Punjab        Gurdaspur     BJP         551177
9             Bihar     Hajipur (Sc)     LJP         539665
10        Karnataka           Hassan   JD(S)         675512
11           Punjab       Hoshiarpur     BJP         416711
12        Telangana        Hyderabad   AIMIM         517239
13            Bihar       Jamui (Sc)     LJP         527860
14            Bihar       

In [42]:
fig = px.bar(winners_low_share, x='general_votes', y='pc_name', color='party', orientation='h', text='general_votes',
             title='Constituencies Elected from Parties with <10% State Vote Share',
             labels={'general_votes':'Votes Received'})
fig.update_layout(yaxis=dict(autorange='reversed'), plot_bgcolor='white', paper_bgcolor='white')
fig.show()


Provides a neutral measure of voter dissatisfaction or protest voting trends.

17. How did NOTA (None of the Above) votes vary across states and constituencies in both 2014 and 2019?


In [61]:
df_all = pd.concat([df_2014, df_2019], ignore_index=True)

df_nota = df_all[df_all['party'].str.upper() == 'NOTA']

nota_state = df_nota.groupby(['year', 'state_name']).agg({
    'general_votes': 'sum'
}).reset_index()

nota_constituency = df_nota.groupby(['year', 'pc_name', 'state_name']).agg({
    'general_votes': 'sum'
}).reset_index()

total_votes_state = df_all.groupby(['year', 'state_name'])['general_votes'].sum().reset_index()
nota_state = nota_state.merge(total_votes_state, on=['year','state_name'])
nota_state['nota_pct'] = (nota_state['general_votes_x'] / nota_state['general_votes_y']) * 100
nota_state.rename(columns={'general_votes_x':'nota_votes','general_votes_y':'total_votes'}, inplace=True)

print("NOTA votes by state:\n", nota_state)
print("\nNOTA votes by constituency:\n", nota_constituency)


NOTA votes by state:
     year                 state_name  nota_votes  total_votes  nota_pct
0   2014  Andaman & Nicobar Islands        1563       190213  0.821710
1   2014             Andhra Pradesh      189101     29796191  0.634648
2   2014          Arunachal Pradesh        6264       580764  1.078579
3   2014                      Assam      147020     15080006  0.974933
4   2014                      Bihar      580865     35870439  1.619342
..   ...                        ...         ...          ...       ...
63  2019                  Telangana      190584     18625876  1.023222
64  2019                    Tripura       23102      2142275  1.078386
65  2019              Uttar Pradesh      723081     86220645  0.838640
66  2019                Uttarakhand       50394      4777548  1.054809
67  2019                West Bengal      546029     57097266  0.956314

[68 rows x 5 columns]

NOTA votes by constituency:
       year          pc_name     state_name  general_votes
0     2014     

In [ ]:
fig = px.bar(
    nota_state,
    x='state_name',
    y='nota_pct',
    color='year',
    barmode='group',
    text='nota_pct',
    title='NOTA (None of the Above) Vote Share by State (2014 vs 2019)',
    color_discrete_sequence=px.colors.qualitative.Bold
)

fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.update_layout(
    yaxis_title='NOTA Vote Share (%)',
    xaxis_title='State',
    height=600,
    xaxis_tickangle=-45
)

fig.show()


Tracks shifts in political influence across regions without subjective commentary on success or failure.

18. Which parties gained the most new constituencies in 2019 compared to 2014?


In [45]:
winners_2014 = df_all[df_all['year']==2014].loc[df_all[df_all['year']==2014].groupby('pc_name')['general_votes'].idxmax()][['pc_name','party']]
winners_2019 = df_all[df_all['year']==2019].loc[df_all[df_all['year']==2019].groupby('pc_name')['general_votes'].idxmax()][['pc_name','party']]
new_wins = winners_2019.merge(winners_2014, on='pc_name', suffixes=('_2019','_2014'))
new_wins['gain'] = new_wins['party_2019'] != new_wins['party_2014']
party_new_wins = new_wins[new_wins['gain']].groupby('party_2019').size().reset_index(name='new_constituencies').sort_values('new_constituencies',ascending=False)
print("\nParties with most new constituencies won in 2019 vs 2014:\n", party_new_wins)


Parties with most new constituencies won in 2019 vs 2014:
    party_2019  new_constituencies
3         BJP                  48
8         INC                  30
7         DMK                  24
24      YSRCP                  14
11      JD(U)                  14
4         BSP                  10
19        SHS                   4
12        JKN                   3
6        CPIM                   3
21         SP                   3
5         CPI                   2
1        AITC                   2
0        ADAL                   2
9         IND                   2
22        TRS                   2
15        NCP                   2
2       AJSUP                   1
16       NDPP                   1
14        MNF                   1
13        LJP                   1
10       IUML                   1
20        SKM                   1
18       RLTP                   1
17        NPF                   1
23        VCK                   1


In [46]:
fig = px.bar(
    party_new_wins,
    x='new_constituencies',
    y='party_2019',
    orientation='h',
    text='new_constituencies',
    title='Parties Gaining Most New Constituencies (2019 vs 2014)',
    color_discrete_sequence=['firebrick'], 
)
fig.update_layout(
    yaxis=dict(autorange='reversed'),
    plot_bgcolor='white',
    paper_bgcolor='white'
)
fig.show()

Identifies stable patterns of engagement, useful for understanding civic participation trends.

19. Which constituencies had consistently high or low voter turnout across both elections?

In [47]:
turnout_pc = df_all.groupby(['pc_name','year'])['voter_turnout_ratio'].mean().reset_index()
pivot_turnout = turnout_pc.pivot(index='pc_name', columns='year', values='voter_turnout_ratio').fillna(0)
pivot_turnout['avg_turnout'] = (pivot_turnout[2014]+pivot_turnout[2019])/2
pivot_turnout['type'] = pd.cut(pivot_turnout['avg_turnout'], bins=[0,50,60,70,100], labels=['Low','Medium','High','Very High'])
consist = pivot_turnout.sort_values('avg_turnout', ascending=False).head(10).reset_index()
print("\nConstituencies with highest average voter turnout (2014 & 2019):\n", consist)


Constituencies with highest average voter turnout (2014 & 2019):
 year              pc_name      2014      2019  avg_turnout type
0                    Tura  0.260341  0.203295     0.231818  Low
1                Nagaland  0.219555  0.165822     0.192689  Low
2          Arunachal East  0.208896  0.144100     0.176498  Low
3             Daman & Diu  0.156014  0.143698     0.149856  Low
4                  Ladakh  0.141553  0.141075     0.141314  Low
5     Autonomous District  0.128942  0.129350     0.129146  Low
6             Lakshadweep  0.123733  0.121683     0.122708  Low
7                 Mizoram  0.154237  0.090066     0.122151  Low
8                Arambagh  0.141842  0.083411     0.112627  Low
9          Chhota Udaipur  0.143271  0.081938     0.112605  Low


In [48]:
unique_colors = px.colors.qualitative.Safe  
colors = unique_colors * (len(consist) // len(unique_colors) + 1)
colors = colors[:len(consist)]

fig = px.bar(
    consist,
    x='avg_turnout',
    y='pc_name',
    orientation='h',
    text='avg_turnout',
    title='Top 10 Constituencies with Highest Average Voter Turnout (2014 & 2019)',
    labels={'avg_turnout':'Average Turnout (%)'},
    color=consist['pc_name'],  
    color_discrete_sequence=colors
)
fig.update_layout(
    yaxis=dict(autorange='reversed'),
    plot_bgcolor='white',
    paper_bgcolor='white'
)
fig.show()

which age groups contributed most to changes in voter turnout helps identify trends in political engagement.

20. Which age groups contributed most to voter turnout changes between 2014 and 2019 ?


In [49]:
df = df[df['age'].notna()]

bins = [18, 25, 35, 45, 55, 65, 100]  
labels = ['18-24', '25-34', '35-44', '45-54', '55-64', '65+']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)

age_turnout = df.groupby(['year', 'age_group']).agg({'general_votes':'sum'}).reset_index()

age_turnout_pivot = age_turnout.pivot(index='age_group', columns='year', values='general_votes').reset_index()
age_turnout_pivot['change'] = age_turnout_pivot[2019] - age_turnout_pivot[2014]

age_turnout_pivot['abs_change'] = age_turnout_pivot['change'].abs()
age_turnout_pivot = age_turnout_pivot.sort_values('abs_change', ascending=False)

print(age_turnout_pivot)

year age_group       2014       2019    change  abs_change
5          65+   84017042  123914230  39897188    39897188
3        45-54  145647881  167723109  22075228    22075228
4        55-64  163183359  184284195  21100836    21100836
2        35-44   88884056   99084042  10199986    10199986
1        25-34   31958040   30364886  -1593154     1593154
0        18-24          0       2424      2424        2424


C:\Users\16052\AppData\Local\Temp\ipykernel_19360\2683770769.py:7: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [50]:
fig = px.pie(
    age_turnout_pivot,
    names='age_group',
    values='abs_change',
    title='Contribution of Age Groups to Voter Turnout Change (2014 vs 2019)',
    color='age_group'
)
fig.update_traces(textinfo='percent+label') 
fig.update_layout(plot_bgcolor='white', paper_bgcolor='white')
fig.show()

21. Which states or constituencies saw the highest increase in youth (18–25) turnout between 2014 and 2019, and how did it correlate with the winning party?

In [63]:
df_all = pd.concat([df_2014, df_2019], ignore_index=True)

youth_votes = df_all[(df_all['age'] >= 18) & (df_all['age'] <= 25)]

# Step 2: Sum youth votes per constituency per year
youth_turnout = youth_votes.groupby(['year','state_name','pc_name']).agg({'general_votes':'sum'}).reset_index()
youth_turnout.rename(columns={'general_votes':'youth_votes'}, inplace=True)

# Step 3: Get total votes per constituency per year
total_votes = df_all.groupby(['year','state_name','pc_name']).agg({'total_votes':'max'}).reset_index()

# Step 4: Merge and calculate youth turnout %
youth_turnout = youth_turnout.merge(total_votes, on=['year','state_name','pc_name'], how='left')
youth_turnout['youth_turnout_pct'] = (youth_turnout['youth_votes'] / youth_turnout['total_votes']) * 100

# Step 5: Pivot to get 2014 and 2019 side by side
pivot_youth = youth_turnout.pivot(index='pc_name', columns='year', values='youth_turnout_pct').reset_index()
pivot_youth.columns.name = None
pivot_youth = pivot_youth.rename(columns={2014:'turnout_2014', 2019:'turnout_2019'})
pivot_youth['youth_turnout_change'] = pivot_youth['turnout_2019'] - pivot_youth['turnout_2014']

# Step 6: Merge winning party info for 2019
winners_2019 = df_all[df_all['year']==2019].sort_values(['pc_name','general_votes'], ascending=[True,False]).groupby('pc_name').first().reset_index()
pivot_youth = pivot_youth.merge(winners_2019[['pc_name','party','state_name']], on='pc_name', how='left')

# Step 7: Top constituencies with highest increase
top_rising = pivot_youth.sort_values('youth_turnout_change', ascending=False).head(10)

# Display results
print("Top 10 constituencies with highest increase in youth turnout (18-25) from 2014→2019:\n")
print(top_rising[['pc_name','state_name','party','turnout_2014','turnout_2019','youth_turnout_change']])


Top 10 constituencies with highest increase in youth turnout (18-25) from 2014→2019:

             pc_name     state_name  party  turnout_2014  turnout_2019  \
100          Tenkasi     Tamil Nadu    DMK      0.418054      0.509077   
87   Purvi Champaran          Bihar    BJP      0.735669      0.757199   
48             Hisar        Haryana    BJP      0.142575      0.127634   
8     Anandpur Sahib         Punjab    INC      0.410773      0.260253   
94   Samastipur (Sc)          Bihar    LJP      2.745182      2.506743   
25         Chandauli  Uttar Pradesh    BJP      1.489611      0.228299   
96      Shahjahanpur  Uttar Pradesh    BJP      2.464142      0.377945   
27            Chatra      Jharkhand    BJP      6.758218      1.370065   
80           Nalanda          Bihar  JD(U)     13.792386      1.102077   
0         Ahmadnagar    Maharashtra    BJP           NaN      0.211591   

     youth_turnout_change  
100              0.091023  
87               0.021531  
48             

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
for i, row in top_rising.iterrows():
    fig.add_trace(go.Scatter(
        x=[row['turnout_2014'], row['turnout_2019']],
        y=[row['pc_name'], row['pc_name']],
        mode='lines+markers',
        marker=dict(
            color=['blue','orange'], 
            size=12
        ),
        line=dict(color='grey', width=2),
        name=row['party'],
        hovertemplate=(
            f"{row['pc_name']} ({row['state_name']})<br>"
            f"2014: {row['turnout_2014']}%<br>"
            f"2019: {row['turnout_2019']}%<br>"
            f"Change: {row['youth_turnout_change']:.2f}%"
        )
    ))

fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers',
                         marker=dict(color='blue', size=12),
                         name='2014 Turnout'))
fig.add_trace(go.Scatter(x=[None], y=[None], mode='markers',
                         marker=dict(color='orange', size=12),
                         name='2019 Turnout'))

fig.update_layout(
    title='Youth (18-25) Turnout per Constituency (2014 vs 2019)',
    xaxis_title='Youth Turnout (%)',
    yaxis_title='Constituency',
    height=600
)

fig.show()
